# Applying Custom Functions to Groups of Data in Pandas

+ Author: Casey Whorton
+ Last Edited: 2/1/2021

## Import Packages

In [223]:
import pandas as pd
import numpy as np
from sklearn import datasets

## Import Toy Dataset

In [224]:
iris = datasets.load_iris()

data = pd.concat([pd.DataFrame(iris.target, columns = ['target']), pd.DataFrame(iris.data, columns = iris.feature_names)], axis = 1)

def target_replace(x):
    for i in [0,1,2]:
        if x == i:
            return(iris.target_names[i])
        
data.target = data.target.apply(target_replace)   

## How Aggregate Functions are Applied to Groups

Typically, a first round analysis might include examining descriptive statistics of your dataset. If you have one table that you are looking at in particular, something like the "describe" method can reveal some key information about your columns such as the number of non-NULL values and the distribution of numerical features. A slightly deeper question to ask is "what are some of these descriptive statistics per group?".  Grouping the data together based on the similar values in a single column and taking the maximum value over all rows in each group looks like this:

In [225]:
data.groupby(['target']).max()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
target,,,,
setosa,5.8,4.4,1.9,0.6
versicolor,7.0,3.4,5.1,1.8
virginica,7.9,3.8,6.9,2.5


The maximum sepal length (in centimeters) is 5.8 for the "setosa" group. This is great but if all we cared about were descriptive statistics at group levels then we could probably program a computer to crank through all the possibilities and return some intersesting findings. What I find comes up more often is the need to apply a custom function to each group in a dataframe. Next, I'll create a custom function that can't be easily duplicated with a groupby aggregate function.

# Define a Custom Function

This could be just about anything. Here, I am interested in the ratio of the range of sepal lengths to petal lengths.  Why?  Maybe I'm interested in seeing which type of flower has greater sepal variation to petal variation. Who knows. The point is that this is a specific question you want answered, and normally you might do a groupby with the max aggregate, then the mins, save those results, and get the ratios. It's possible, but why not execute a custom function over the groups? We're not limited to aggregates, you could find the 2nd value in a dataframe's column if you wanted.

In [226]:
def find_ratio(df):
    """
    Calculates the ratio of the sepal length range to the petal length range for a pandas dataframe.
    
    Arguments:
    ----------
        df (pandas dataframe): a dataframe to calculate the ratio over
        
    Returns:
    --------
        ratio (numpy float): the calculated ratio
    """
    ratio = (df['sepal length (cm)'].max() - df['sepal length (cm)'].min())/(df['petal length (cm)'].max() - df['petal length (cm)'].min())
    
    return(ratio)

## Checkout The Groupby Object

In [227]:
#for i in data.groupby(['target']):
#    print(i)

Notice that every element in the groupby is a tuple, with the first element containing whatever you grouped the data by (this can be more than one column, in which case another tuple) and the grouped data (a pandas dataframe). This means that whatever function you use on the groupby elements needs to take both the groups and dataframes.

## The First Way

Simply use the apply method to each dataframe in the groupby object. This is the most straightforward way and the easiest to understand.  Notice that the function takes a dataframe as its only argument, so any code within the custom function needs to work on a pandas dataframe.

In [228]:
data.groupby(['target']).apply(find_ratio)

target
setosa        1.666667
versicolor    1.000000
virginica     1.250000
dtype: float64

It is also possible to accomplish this with a lambda function, executed over each dataframe.

In [229]:
data.groupby(['target']).apply(lambda x: find_ratio(x))

target
setosa        1.666667
versicolor    1.000000
virginica     1.250000
dtype: float64

## The Second Way

Save the groupby element and execute the function for every element in the groupby object. This second option takes a little more work, but could be what you are looking for in terms of customization. While we applied our custom function to each dataframe directly before, we will need to rewrite the function slightly to accept the groupby element as the argument. But first, create a groupby object for the column(s) you want to groupby and assign it a variable name. Next, rewrite the function to work on each groupby in the groupby element. Note: a groupby object is iterable (meaning python can loop through it) and contains both the levels of the grouping and the resulting dataframe.

In [230]:
# We will call the groupby object "grouping"

grouping = data.groupby(['target'])

In [231]:
grouping

In [232]:
for i in grouping:
    print(i[0])

setosa
versicolor
virginica


In [233]:
for i in grouping:
    print(i[1].head(2))

   target  sepal length (cm)  sepal width (cm)  petal length (cm)  \
0  setosa                5.1               3.5                1.4   
1  setosa                4.9               3.0                1.4   

   petal width (cm)  
0               0.2  
1               0.2  
        target  sepal length (cm)  sepal width (cm)  petal length (cm)  \
50  versicolor                7.0               3.2                4.7   
51  versicolor                6.4               3.2                4.5   

    petal width (cm)  
50               1.4  
51               1.5  
        target  sepal length (cm)  sepal width (cm)  petal length (cm)  \
100  virginica                6.3               3.3                6.0   
101  virginica                5.8               2.7                5.1   

     petal width (cm)  
100               2.5  
101               1.9  


In [234]:
def find_ratio_for_group(grouping):
    """
    Calculates the ratio of the sepal length range to the petal length range for a pandas groupby element.
    
    Arguments:
    ----------
        grouping (pandas groupby): a groupby element to calculate the ratio over
        
    Returns:
    --------
        {group_label: ratio} (dict): a dictionary of the group label and calculated ratio
    """
    
    (group_label, df) = grouping # in our case we know it is a single column, but this could be used on multiple columns as well
    
    ratio = (df['sepal length (cm)'].max() - df['sepal length (cm)'].min())/(df['petal length (cm)'].max() - df['petal length (cm)'].min())
    
    return({group_label: ratio})

Execute the function over the groupby element in a loop or using a list comprehension. I'm choosing to update a dictionary in the first example and to append to a list in the second example. Each method has its advantages.

In [235]:
# Using a dictionary

mylist = dict()

for i in grouping:
    
    mylist.update(find_ratio_for_group(i))
    
pd.DataFrame(data = mylist.values(), index = mylist.keys(), columns = ['ratio'])

# Using a list comprehension

ratios = [find_ratio_for_group(i).values() for i in grouping]

pd.DataFrame(data = ratios, index = grouping.groups, columns = ['ratio'])

,ratio
setosa,1.666667
versicolor,1.000000
virginica,1.250000


# References

+ https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html

# END